<a href="https://colab.research.google.com/github/Nikhil-Rao-789/Ai-for-heath/blob/main/Scripts/vis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.backends.backend_pdf import PdfPages

from google.colab import drive
drive.mount('/content/drive')

PROJECT_ROOT = "/content/drive/MyDrive/AI for Health"
DATA_DIR = f"{PROJECT_ROOT}/Data"
VIS_DIR = f"{PROJECT_ROOT}/Visualizations"


Mounted at /content/drive


In [ ]:
participant = input("Enter Participant name for Visualizations: ")
dfSP02 = pd.read_csv(f'{DATA_DIR}/{participant}/SPO2.txt', sep=';',skiprows=7,names=["Time","Value"])
dfFlow = pd.read_csv(f'{DATA_DIR}/{participant}/Flow.txt', sep=';',skiprows=7,names=["Time","Value"])
dfThorac = pd.read_csv(f'{DATA_DIR}/{participant}/Thorac.txt', sep=';',skiprows=7,names=["Time","Value"])
dfFlowEvents = pd.read_csv(f'{DATA_DIR}/{participant}/FlowEvents.txt', sep=';',skiprows=5,names=["Range","Duration","Event","Stage"])

Enter Participant name for Visualizations: AP01


In [ ]:
dfSP02["Time"] = pd.to_datetime(dfSP02["Time"],format="%d.%m.%Y %H:%M:%S,%f")
dfFlow["Time"] = pd.to_datetime(dfFlow["Time"],format="%d.%m.%Y %H:%M:%S,%f")
dfThorac["Time"] = pd.to_datetime(dfThorac["Time"],format="%d.%m.%Y %H:%M:%S,%f")

dfFlowEvents[["Start","End"]] = dfFlowEvents["Range"].str.split("-", expand=True)
dfFlowEvents["Date"] = dfFlowEvents["Start"].str.split().str[0]
dfFlowEvents["Start"] = pd.to_datetime(dfFlowEvents["Start"],format="%d.%m.%Y %H:%M:%S,%f")
dfFlowEvents["End"] = pd.to_datetime(dfFlowEvents["Date"] + " " + dfFlowEvents["End"],format="%d.%m.%Y %H:%M:%S,%f")

In [ ]:
dfSP02.set_index("Time",inplace = True)
dfFlow.set_index("Time",inplace = True)
dfThorac.set_index("Time",inplace = True)

In [ ]:
minSPO2 = 80
maxSPO2 = 100
minFlow = 70
maxFlow = 130
minThorac = 5
maxThorac = 25

dfSP02["Value"] = dfSP02["Value"].where((dfSP02["Value"] >= minSPO2) & (dfSP02["Value"] <= maxSPO2), np.nan).interpolate(method="linear")
dfFlow["Value"] = dfFlow["Value"].where((dfFlow["Value"] >= minFlow) & (dfFlow["Value"] <= maxFlow), np.nan).interpolate(method="linear")
dfThorac["Value"] = dfThorac["Value"].where((dfThorac["Value"] >= minThorac) & (dfThorac["Value"] <= maxThorac), np.nan).interpolate(method="linear")

target_freq = "31.25ms"

dfSP02 = (dfSP02.resample(target_freq).interpolate(method="time"))
dfFlow = dfFlow.resample(target_freq).mean().interpolate("time")
dfThorac = dfThorac.resample(target_freq).mean().interpolate("time")

In [ ]:
start = max(dfSP02.index[0],dfFlow.index[0],dfThorac.index[0])
end = min(dfSP02.index[-1],dfFlow.index[-1],dfThorac.index[-1])

window = pd.Timedelta(minutes=5)
ranges = pd.date_range(start=start,end=end,freq=window)

In [ ]:
pdf_path = f"{VIS_DIR}/{participant}_report.pdf"

with PdfPages(pdf_path) as pdf:

    for start in ranges:

        end = start + window

        dfSP02_5min = dfSP02.loc[start:end]
        dfFlow_5min = dfFlow.loc[start:end]
        dfThorac_5min = dfThorac.loc[start:end]
        events_5min = dfFlowEvents[(dfFlowEvents["Start"] <= end) &(dfFlowEvents["End"] >= start)
]

        fig, ax = plt.subplots(3, 1,figsize=(18,8),sharex=True,constrained_layout=True)

        fig.patch.set_facecolor("white")
        fig.patch.set_edgecolor("black")
        fig.patch.set_linewidth(1.2)

        ax[0].plot(dfFlow_5min.index, dfFlow_5min["Value"],color='tab:blue',label = "Nasal Flow")
        ax[0].legend(loc = "upper right")
        ax[0].set_ylabel("Nasal Flow (L/min)")
        for _, event in events_5min.iterrows():
          start_e = max(event["Start"], start)
          end_e   = min(event["End"], end)
          ax[0].axvspan(start_e,end_e,color="palegoldenrod",alpha=0.35,zorder=0)
          mid_time = start_e + (end_e - start_e) / 2
          y_pos = ax[0].get_ylim()[1] * 0.85
          ax[0].text(mid_time,y_pos,event["Event"],ha="center",va="center",fontsize=9,color="black",bbox=dict(facecolor="white",alpha=0.6,edgecolor="none",boxstyle="round,pad=0.2"))

        ax[1].plot(dfThorac_5min.index, dfThorac_5min["Value"],color='tab:orange',label = "Thoracic/Abdominal Resp.")
        ax[1].legend(loc = "upper right")
        ax[1].set_ylabel("Resp. Amplitude")

        ax[2].plot(dfSP02_5min.index, dfSP02_5min["Value"],color='grey',label = "SpO₂")
        ax[2].legend(loc = "upper right")
        ax[2].set_ylabel("SpO2 (%)")

        ax[2].xaxis.set_major_formatter(mdates.DateFormatter("%d %H:%M:%S"))
        ax[2].xaxis.set_major_locator(mdates.SecondLocator(interval=5))
        plt.setp(ax[2].get_xticklabels(), rotation=90)
        fig.supxlabel("Time")

        for a in ax:
            a.grid(True, color="lightgray")
            for spine in a.spines.values():
                spine.set_color("black")
                spine.set_linewidth(1)

        fig.suptitle(f"{participant} - {start:%Y-%m-%d %H:%M} to {end:%Y-%m-%d %H:%M}",fontsize=14)

        pdf.savefig(fig)
        plt.close(fig)